In [1]:
import PyPDF2
import re
import glob
import pandas as pd
import numpy as np

In [39]:
path = r'C:\\DATASETS\\Empresas Baixadas\\pdf'
all_files = glob.glob(path + "/*.pdf")

empresas_baixadas_df = pd.DataFrame(columns=['Arquivo', 'CNPJ'])

for filename in all_files:
    print(filename)
    pdfReader = PyPDF2.PdfFileReader(filename)
    count = pdfReader.numPages
    # Percorre cada página do PDF:
    for i in range(count):
        page = pdfReader.getPage(i)
        output = page.extractText()
        # Procura por CNPJs:
        cnpj_search = re.findall( "(?:\d ?){2}\.?(?:\d ?){3}.?(?:\d ?){3}.?\d[\s\S]?\d[\s\S]?\d[\s\S]?\d[\s\S]?\-?\n?\d[\s\S]?\d", output)

        print('Página: ', i, ' - Qte cnpj: ', len(cnpj_search))
        
        cnpj_search = [cnpj.replace('\n', '') for cnpj in cnpj_search]
        
        temp_df = pd.DataFrame(columns=['Arquivo', 'CNPJ'])
        temp_df.CNPJ = cnpj_search
        temp_df.Arquivo = filename.rsplit('\\',1)[1]
    
        empresas_baixadas_df = pd.concat([empresas_baixadas_df, temp_df], ignore_index=True)    
        
empresas_baixadas_df

C:\\DATASETS\\Empresas Baixadas\\pdf\ADE Cocad n 2 de 2018 - Edital Intimação n 1 de 2018 - Inexistencia de Fato - noteiras-ocas.pdf
Página:  0  - Qte cnpj:  39
Página:  1  - Qte cnpj:  39
Página:  2  - Qte cnpj:  39
Página:  3  - Qte cnpj:  39
Página:  4  - Qte cnpj:  39
Página:  5  - Qte cnpj:  39
Página:  6  - Qte cnpj:  39
Página:  7  - Qte cnpj:  19
Página:  8  - Qte cnpj:  0
Página:  9  - Qte cnpj:  0
Página:  10  - Qte cnpj:  0
Página:  11  - Qte cnpj:  0
Página:  12  - Qte cnpj:  0
Página:  13  - Qte cnpj:  0
Página:  14  - Qte cnpj:  1
C:\\DATASETS\\Empresas Baixadas\\pdf\ADE Cocad n 2 de 2020 - anexo - Edital Intimacao n 1 de 2019 - Inexistencia de Fato.pdf
Página:  0  - Qte cnpj:  41
Página:  1  - Qte cnpj:  44
Página:  2  - Qte cnpj:  43
Página:  3  - Qte cnpj:  43
Página:  4  - Qte cnpj:  24
C:\\DATASETS\\Empresas Baixadas\\pdf\ADE Cocad n 3 de 2018 - distribuidoras - Edital Intimação n 2 de 2018 - Inexistencia de Fato.pdf
Página:  0  - Qte cnpj:  39
Página:  1  - Qte cnpj

,Arquivo,CNPJ
0,ADE Cocad n 2 de 2018 - Edital Intimação n 1 d...,00903551000165
1,ADE Cocad n 2 de 2018 - Edital Intimação n 1 d...,01005247000163
2,ADE Cocad n 2 de 2018 - Edital Intimação n 1 d...,01291703000189
3,ADE Cocad n 2 de 2018 - Edital Intimação n 1 d...,01424175000199
4,ADE Cocad n 2 de 2018 - Edital Intimação n 1 d...,01424175000270
...,...,...
2478,lista-baixa-20-02-2020 - Edital Cocad 1 de 202...,30.802.439/0001-07
2479,lista-baixa-20-02-2020 - Edital Cocad 1 de 202...,31.429.447/0001-04
2480,lista-baixa-20-02-2020 - Edital Cocad 1 de 202...,31.980.500/0001-60
2481,lista-baixa-20-02-2020 - Edital Cocad 1 de 202...,28.246.970/0001-90


In [40]:
empresas_baixadas_df.CNPJ.nunique()

2482

In [41]:
empresas_baixadas_df.Arquivo.value_counts()

ADE Cocad n 3 de 2018 - distribuidoras - Edital Intimação n 2 de 2018 - Inexistencia de Fato.pdf    1938
ADE Cocad n 2 de 2018 - Edital Intimação n 1 de 2018 - Inexistencia de Fato - noteiras-ocas.pdf      293
ADE Cocad n 2 de 2020 - anexo - Edital Intimacao n 1 de 2019 - Inexistencia de Fato.pdf              195
lista-baixa-20-02-2020 - Edital Cocad 1 de 2020 - Inexistência de Fato.pdf                            31
ADE Cocad n 3 de 2020 - anexo - sem o edital.pdf                                                      26
Name: Arquivo, dtype: int64

In [42]:
empresas_baixadas_df.CNPJ.str.len().value_counts()

14    2228
18     252
17       3
Name: CNPJ, dtype: int64

In [43]:
empresas_baixadas_df[empresas_baixadas_df.CNPJ.str.len()==17]

,Arquivo,CNPJ
1790,ADE Cocad n 3 de 2018 - distribuidoras - Edita...,01548235000185G10
1791,ADE Cocad n 3 de 2018 - distribuidoras - Edita...,01548235000266G10
1792,ADE Cocad n 3 de 2018 - distribuidoras - Edita...,01548235000428G10


In [47]:
# Corrigindo manualmente os 3 registros errados, mostrados acima:
empresas_baixadas_df.loc[1790].CNPJ = '01548235000185'
empresas_baixadas_df.loc[1791].CNPJ = '01548235000266'
empresas_baixadas_df.loc[1792].CNPJ = '01548235000428'

##### Importação dos ADE que não foram passíveis de leitura em PDF (foram então convertidos em TXT):

In [49]:
empresas_baixadas_txt_df = pd.DataFrame(columns=['Arquivo', 'CNPJ'])

path = r'C:\\DATASETS\\Empresas Baixadas\\txt'
all_files = glob.glob(path + "/*.txt")

for filename in all_files:
    print(filename)
    txt = open(filename, "r")

    cnpj_search = re.findall( "\d{2}\.\d{3}\.\d{3}\/\d{4}\-\d{2}", txt.read())

    print('Qte cnpj: ', len(cnpj_search))
    
    temp_df = pd.DataFrame(columns=['Arquivo', 'CNPJ'])
    temp_df.CNPJ = cnpj_search
    temp_df.Arquivo = filename.rsplit('\\',1)[1]

    empresas_baixadas_txt_df = pd.concat([empresas_baixadas_txt_df, temp_df], ignore_index=True)    

empresas_baixadas_txt_df

C:\\DATASETS\\Empresas Baixadas\\txt\ADE Cocad n 1 de 2018 - SN - registro cancelado - omisso - inadimplente.txt
Qte cnpj:  1372246
C:\\DATASETS\\Empresas Baixadas\\txt\lista-de-cnpj-baixados-em-03-04-e-06-04-2020 - sem edital.txt
Qte cnpj:  69


,Arquivo,CNPJ
0,ADE Cocad n 1 de 2018 - SN - registro cancelad...,00.000.942/0001-70
1,ADE Cocad n 1 de 2018 - SN - registro cancelad...,13.762.046/0001-90
2,ADE Cocad n 1 de 2018 - SN - registro cancelad...,16.577.209/0001-52
3,ADE Cocad n 1 de 2018 - SN - registro cancelad...,19.071.487/0001-21
4,ADE Cocad n 1 de 2018 - SN - registro cancelad...,00.001.102/0001-21
...,...,...
1372310,lista-de-cnpj-baixados-em-03-04-e-06-04-2020 -...,35.007.721/0001-08
1372311,lista-de-cnpj-baixados-em-03-04-e-06-04-2020 -...,34.665.501/0001-09
1372312,lista-de-cnpj-baixados-em-03-04-e-06-04-2020 -...,34.670.798/0001-09
1372313,lista-de-cnpj-baixados-em-03-04-e-06-04-2020 -...,33.930.020/0001-10


In [50]:
empresas_baixadas_txt_df.CNPJ.nunique()

1372315

In [51]:
empresas_baixadas_txt_df.Arquivo.value_counts()

ADE Cocad n 1 de 2018 - SN - registro cancelado - omisso - inadimplente.txt    1372246
lista-de-cnpj-baixados-em-03-04-e-06-04-2020 - sem edital.txt                       69
Name: Arquivo, dtype: int64

In [52]:
empresas_baixadas_txt_df.CNPJ.str.len().value_counts()

18    1372315
Name: CNPJ, dtype: int64

In [53]:
empresas_baixadas_df = pd.concat([empresas_baixadas_df, empresas_baixadas_txt_df], ignore_index=True)
empresas_baixadas_df

,Arquivo,CNPJ
0,ADE Cocad n 2 de 2018 - Edital Intimação n 1 d...,00903551000165
1,ADE Cocad n 2 de 2018 - Edital Intimação n 1 d...,01005247000163
2,ADE Cocad n 2 de 2018 - Edital Intimação n 1 d...,01291703000189
3,ADE Cocad n 2 de 2018 - Edital Intimação n 1 d...,01424175000199
4,ADE Cocad n 2 de 2018 - Edital Intimação n 1 d...,01424175000270
...,...,...
1374793,lista-de-cnpj-baixados-em-03-04-e-06-04-2020 -...,35.007.721/0001-08
1374794,lista-de-cnpj-baixados-em-03-04-e-06-04-2020 -...,34.665.501/0001-09
1374795,lista-de-cnpj-baixados-em-03-04-e-06-04-2020 -...,34.670.798/0001-09
1374796,lista-de-cnpj-baixados-em-03-04-e-06-04-2020 -...,33.930.020/0001-10


In [54]:
empresas_baixadas_df.Arquivo.value_counts()

ADE Cocad n 1 de 2018 - SN - registro cancelado - omisso - inadimplente.txt                         1372246
ADE Cocad n 3 de 2018 - distribuidoras - Edital Intimação n 2 de 2018 - Inexistencia de Fato.pdf       1938
ADE Cocad n 2 de 2018 - Edital Intimação n 1 de 2018 - Inexistencia de Fato - noteiras-ocas.pdf         293
ADE Cocad n 2 de 2020 - anexo - Edital Intimacao n 1 de 2019 - Inexistencia de Fato.pdf                 195
lista-de-cnpj-baixados-em-03-04-e-06-04-2020 - sem edital.txt                                            69
lista-baixa-20-02-2020 - Edital Cocad 1 de 2020 - Inexistência de Fato.pdf                               31
ADE Cocad n 3 de 2020 - anexo - sem o edital.pdf                                                         26
Name: Arquivo, dtype: int64

In [55]:
empresas_baixadas_df.CNPJ.str.len().value_counts()

18    1372567
14       2231
Name: CNPJ, dtype: int64

In [56]:
empresas_baixadas_df.CNPJ = empresas_baixadas_df.CNPJ.str.replace('[\.\-\/]', '', regex=True)

In [57]:
empresas_baixadas_df.CNPJ.str.len().value_counts()

14    1374798
Name: CNPJ, dtype: int64

In [58]:
empresas_baixadas_df.CNPJ.nunique()

1374797

In [60]:
empresas_baixadas_df = empresas_baixadas_df.drop_duplicates(subset=['CNPJ'])

In [61]:
empresas_baixadas_df

,Arquivo,CNPJ
0,ADE Cocad n 2 de 2018 - Edital Intimação n 1 d...,00903551000165
1,ADE Cocad n 2 de 2018 - Edital Intimação n 1 d...,01005247000163
2,ADE Cocad n 2 de 2018 - Edital Intimação n 1 d...,01291703000189
3,ADE Cocad n 2 de 2018 - Edital Intimação n 1 d...,01424175000199
4,ADE Cocad n 2 de 2018 - Edital Intimação n 1 d...,01424175000270
...,...,...
1374793,lista-de-cnpj-baixados-em-03-04-e-06-04-2020 -...,35007721000108
1374794,lista-de-cnpj-baixados-em-03-04-e-06-04-2020 -...,34665501000109
1374795,lista-de-cnpj-baixados-em-03-04-e-06-04-2020 -...,34670798000109
1374796,lista-de-cnpj-baixados-em-03-04-e-06-04-2020 -...,33930020000110


In [62]:
# Salvando resultado:
empresas_baixadas_df.to_csv('C:\DATASETS\Empresas Baixadas\empresas_baixadas_extraido_arquivos_ADE.csv', index=False)